In [3]:
from testing.test_suite import run_all_tests

run_all_tests()

[2023/11/27 11:40:08] ppocr DEBUG: Namespace(help='==SUPPRESS==', use_gpu=False, use_xpu=False, use_npu=False, ir_optim=True, use_tensorrt=False, min_subgraph_size=15, precision='fp32', gpu_mem=500, gpu_id=0, image_dir=None, page_num=0, det_algorithm='DB', det_model_dir='/Users/leviharris/.paddleocr/whl/det/en/en_PP-OCRv3_det_infer', det_limit_side_len=960, det_limit_type='max', det_box_type='quad', det_db_thresh=0.3, det_db_box_thresh=0.6, det_db_unclip_ratio=1.5, max_batch_size=10, use_dilation=False, det_db_score_mode='fast', det_east_score_thresh=0.8, det_east_cover_thresh=0.1, det_east_nms_thresh=0.2, det_sast_score_thresh=0.5, det_sast_nms_thresh=0.2, det_pse_thresh=0, det_pse_box_thresh=0.85, det_pse_min_area=16, det_pse_scale=1, scales=[8, 16, 32], alpha=1.0, beta=1.0, fourier_degree=5, rec_algorithm='SVTR_LCNet', rec_model_dir='/Users/leviharris/.paddleocr/whl/rec/en/en_PP-OCRv4_rec_infer', rec_image_inverse=True, rec_image_shape='3, 48, 320', rec_batch_num=6, max_text_length=

test_blank_regex (testing.test_suite.RegexTests) ... ok
test_regex_edge_cases (testing.test_suite.RegexTests) ... ok
test_regex_no_match (testing.test_suite.RegexTests) ... ok
test_regex_with_spaces (testing.test_suite.RegexTests) ... ok
test_time_coversion_blank (testing.test_suite.TimeConversionTests) ... ok
test_time_coversion_standard (testing.test_suite.TimeConversionTests) ... ok
test_time_rare_formatting (testing.test_suite.TimeConversionTests) ... ok
test_extract_text_with_paddle_blank (testing.test_suite.TextExtractionTests) ... 

In [ ]:
from moviepy.editor import VideoFileClip

def extract_clip(video_path, start_time, output_path):
    """
    Extracts a 10-second clip from the specified video starting at `start_time`.
    
    :param video_path: Path to the source video file.
    :param start_time: Start time for the clip in seconds.
    :param output_path: Path where the extracted clip will be saved.
    """
    try:
        video = VideoFileClip(video_path)
        end_time = start_time + 10
        clip = video.subclip(start_time, end_time)
        clip.write_videofile(output_path, codec="libx264")
        print(f"Clip successfully extracted to {output_path}")
    except Exception as e:
        print(f"An error occurred: {e}")

in_path = r"demo\17590_01-23-2016_2_Golden State Warriors_9_Indiana Pacers_period4.mp4"
out_path = r"demo/demo_17590_period_4.mp4"
extract_clip(in_path, 60, out_path)  # Extracts a clip starting at 1 minute


In [ ]:
from temporal_grounding import extract_timestamps_from_video

vp = r"demo\demo_17590_period_4.mp4"
op = "demo/timestamps.json"
extract_timestamps_from_video(vp, op)

# TODO
# 1. Post Process Timestamps
# 2. Map time-remaining to stavulog
# 3. visualize results

In [ ]:
testing/assets/blank_images/black.png

In [ ]:
from ultralytics import YOLO
from paddleocr import PaddleOCR

MODEL_PATH = r"models/yolo/weights/tr_roi_finetune_130_large.pt"


class Models:

    yolo = yolo = YOLO(MODEL_PATH)
    paddle_ocr = PaddleOCR(use_angle_cls=True,
                           lang='en',
                           show_log=False,
                           det_db_score_mode='slow',
                           ocr_version='PP-OCRv4',
                           rec_algorithm='SVTR_LCNet',
                           drop_score=0.8,
                           )


In [1]:
import cv2
import torch
import json
import os
import re
import time
import numpy as np

from tqdm import tqdm
from PIL import Image
from typing import List

from viz import visualize_timestamps
from utilities.models import Models
from utilities.constants import *

def extract_text_with_paddle(image) -> List[str]:
    """
    Returns a [str] containing all words found in a
    provided PIL image.
    """

    def resize_with_opencv(image_pil, new_size):
        image_np = np.array(image_pil)
        image_np = cv2.cvtColor(image_np, cv2.COLOR_RGB2BGR)
        resized_image = cv2.resize(image_np, new_size)
        return resized_image

    if image is None:
        return []
    image = Image.fromarray(image)
    image = image.convert("RGB")
    ideal_height = 100
    scale_factor = ideal_height / image.height
    new_size = (int(image.width * scale_factor),
                int(image.height * scale_factor))
    image = resize_with_opencv(image, new_size)
    results = []
    raw_result = Models.paddle_ocr.ocr(image, det=False, rec=False, cls=False)
    text_arr = raw_result[1]
    for pred in text_arr:
        word = pred[0]
        results.append(word)
    return results

/Users/leviharris/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


[2023/11/27 11:42:13] ppocr DEBUG: Namespace(help='==SUPPRESS==', use_gpu=False, use_xpu=False, use_npu=False, ir_optim=True, use_tensorrt=False, min_subgraph_size=15, precision='fp32', gpu_mem=500, gpu_id=0, image_dir=None, page_num=0, det_algorithm='DB', det_model_dir='/Users/leviharris/.paddleocr/whl/det/en/en_PP-OCRv3_det_infer', det_limit_side_len=960, det_limit_type='max', det_box_type='quad', det_db_thresh=0.3, det_db_box_thresh=0.6, det_db_unclip_ratio=1.5, max_batch_size=10, use_dilation=False, det_db_score_mode='fast', det_east_score_thresh=0.8, det_east_cover_thresh=0.1, det_east_nms_thresh=0.2, det_sast_score_thresh=0.5, det_sast_nms_thresh=0.2, det_pse_thresh=0, det_pse_box_thresh=0.85, det_pse_min_area=16, det_pse_scale=1, scales=[8, 16, 32], alpha=1.0, beta=1.0, fourier_degree=5, rec_algorithm='SVTR_LCNet', rec_model_dir='/Users/leviharris/.paddleocr/whl/rec/en/en_PP-OCRv4_rec_infer', rec_image_inverse=True, rec_image_shape='3, 48, 320', rec_batch_num=6, max_text_length=

In [3]:
print("read")
image = cv2.imread(r"/Users/leviharris/Documents/contextualized-shot-quality-estimation/testing/assets/example_cropped_rois/time_remaining/time_remaining_1.png")
results = extract_text_with_paddle(image)
assert results == ["2:41"]

In [ ]:
import os

os.path.isfile('testing/assets/example_cropped_rois/time_remaining/time_remaining_1.png')

In [ ]:
import cv2